In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pickle
from web3 import Web3, HTTPProvider
import json
import web3
import ipfs_api as ip
import os
import time 
from imblearn.under_sampling import RandomUnderSampler


In [13]:
NUM_EPOCHS = 20

class Client_model:
    def __init__(self, data_path, user_id,client_index):
        self.data_path = data_path
        self.user_id = user_id
        self.client_index  = client_index
        self.abi = abi
        x = ml_contract.functions.getModelArch().call()
        x = json.loads(x)
        self.trained_model  = tf.keras.models.model_from_json(x)

    def load_data(self):
        dataset = pd.read_csv(self.data_path, index_col=0)
        
        rus = RandomUnderSampler(random_state=42)
        x_ip, Y = rus.fit_resample(dataset.drop(columns=['ip.src_host', 'Attack_label', 'Attack_type']), dataset['Attack_type'])

        encoder = OneHotEncoder(sparse_output=False)
        encoder = encoder.fit(np.array(dataset['Attack_type'].unique()).reshape(-1, 1))
        # x_ip = dataset[dataset['ip.src_host'] == self.user_id].copy()
        # Y = x_ip['Attack_type']
        Y = encoder.transform(np.array(Y).reshape(-1, 1))
        # x_ip.drop(columns=['ip.src_host', 'Attack_label', 'Attack_type'], inplace=True)
        int_data = x_ip.select_dtypes(include=int).astype(float).to_numpy()
        float_data = x_ip.select_dtypes(include=float).to_numpy()
        obj_data = x_ip.select_dtypes(include=bool).astype(float).to_numpy()
        x_ip = np.concatenate([float_data, int_data, obj_data], axis=1)
        self.x_train_user, self.x_test_user, self.y_train_user, self.y_test_user = train_test_split(
            x_ip, Y, test_size=.2, random_state=42)

    def preprocess(self):
        BATCH_SIZE = 20
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER = 10
        self.load_data()
        self.x_train_user = tf.convert_to_tensor(self.x_train_user)
        data_size = len(self.x_train_user)
        print(len(self.x_train_user))
        self.x_test_user = tf.convert_to_tensor(self.x_test_user)
        self.y_train_user = tf.convert_to_tensor(self.y_train_user)
        self.y_test_user = tf.convert_to_tensor(self.y_test_user)
        self.x_train_user = tf.reshape(self.x_train_user, [-1, 46])
        self.x_test_user = tf.reshape(self.x_test_user, [-1, 46])
        self.y_train_user = tf.reshape(self.y_train_user, [-1, 15])
        self.y_test_user = tf.reshape(self.y_test_user, [-1, 15])
        self.train_dataset = tf.data.Dataset.from_tensor_slices((self.x_train_user, self.y_train_user))
        self.test_dataset = tf.data.Dataset.from_tensor_slices((self.x_test_user, self.y_test_user))
        self.train_dataset = self.train_dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).prefetch(
            PREFETCH_BUFFER)
        self.test_dataset = self.test_dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).prefetch(
            PREFETCH_BUFFER)
        ml_contract.functions.setDataLen(self.client_index, data_size).transact({'from':w3.eth.accounts[self.client_index]})

    # def get_model_arch(self):
    #     x = ml_contract.functions.getModelArch().call()
    #     x = json.loads(x)
    #     self.trained_model  = tf.keras.models.model_from_json(x)
    #     # print(x,type(x))
    #     # self.trained_model.summary()
    
    def get_model_weights(self):
        ###############################################
        ##### Get Global Model weights ################
        global_weights_cid = ml_contract.functions.getGlobalModel().call()
        print(global_weights_cid)
        if os.path.exists(f'./tmp/{global_weights_cid}'):
            os.remove(f'./tmp/{global_weights_cid}')
        ip.download(global_weights_cid,'./tmp/')
        ### Load  file 
        if os.path.exists(f'./tmp/client_{self.client_index}.weights.h5') : 
            os.remove(f'./tmp/client_{self.client_index}.weights.h5')
        os.rename(f'./tmp/{global_weights_cid}', f'./tmp/client_{self.client_index}.weights.h5')
        self.trained_model.load_weights(f'./tmp/client_{self.client_index}.weights.h5')
        #remove file after loading it 
        os.remove(f'./tmp/client_{self.client_index}.weights.h5')
    
    def send_model_hash(self):

        ################################
        ##### save model weights ####### 
        self.trained_model.save_weights(f'./tmp/client_{self.client_index}.weights.h5')
        print("wieghtd  ",f'./tmp/client_{self.client_index}.weights.h5')
        ################################
        ##### send model hash ##########
        cid = ip.publish(f'./tmp/client_{self.client_index}.weights.h5')
        print(cid)
        ml_contract.functions.SetModelHash(self.client_index, cid).transact({'from':w3.eth.accounts[self.client_index]})
        #### Remove weights 
        os.remove(f'./tmp/client_{self.client_index}.weights.h5')
    
    def train_model(self):
        print(f'Client {self.user_id} Training Started')
        self.preprocess()
        self.get_model_weights()
        self.trained_model.compile(
            optimizer=tf.keras.optimizers.SGD(),
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
            metrics=[
                tf.keras.metrics.Accuracy(),
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall(),
                tf.keras.metrics.AUC()
            ]
        )
        self.trained_model.fit(self.train_dataset, epochs=NUM_EPOCHS)
        # self.train_model.summary()
        ### save model 
        ml_contract.functions.setNewModelReady(self.client_index).transact({'from':w3.eth.accounts[self.client_index]})
        print(f'Client {self.user_id} Training Finished')
        self.send_model_hash()
        print(f'Client {self.user_id} Weights Sent')
        print("_____________________________________________________________")
        
        


In [14]:
w3 = web3.Web3(web3.HTTPProvider('http://127.0.0.1:7545'))
### Contract Address
contract_address = '0x1c54d379603F38986B9c6c2F8216076F4324e24d'
######
abi = '[{"constant":true,"inputs":[],"name":"loop","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"serverNewModel","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"isStartTraining","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"model_architecture","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"clientCount","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"addressToIndex","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"uint256"}],"name":"clients","outputs":[{"name":"client_index","type":"uint256"},{"name":"new_model_hash","type":"string"},{"name":"new_model_ready","type":"bool"},{"name":"data_index","type":"string"},{"name":"data_len","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_state","type":"bool"}],"name":"setTrainLoop","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"getTrainLoop","outputs":[{"name":"loop","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"model","type":"string"}],"name":"setGlobalModel","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"getGlobalModel","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_data_index","type":"string"}],"name":"addClient","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"clientIndex","type":"uint256"},{"name":"newDataLen","type":"uint256"}],"name":"setDataLen","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"clientIndex","type":"uint256"}],"name":"getDataLen","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"sumDataLen","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"clientIndex","type":"uint256"},{"name":"modelHash","type":"string"}],"name":"SetModelHash","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"clientIndex","type":"uint256"}],"name":"getModelHash","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"clientIndex","type":"uint256"},{"name":"newModel","type":"string"}],"name":"updateModel","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"jsonString","type":"string"}],"name":"sendModelArch","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getModelArch","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_start","type":"bool"}],"name":"setIsStartTraining","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getIsStartTraining","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"clientIndex","type":"uint256"}],"name":"setNewModelReady","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"clientIndex","type":"uint256"}],"name":"clearNewModelReady","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]'

### Data Path 
data_path = './Cleaned_data.csv'
### connect to contract
ml_contract = w3.eth.contract(abi=abi, address=contract_address)



In [ ]:
users = [] ### List holds ips sent from server 
clients_obj = []  ### List holds obj from client class

###############################################
####### Get data from server ##################
i = 0 
while i < (ml_contract.functions.clientCount().call()) :
    users.append(ml_contract.functions.clients(i).call())
    i+=1


###############################################
############# Start clients ################### 
for i,j in enumerate(users):
    clients_obj.append(Client_model(data_path,j[3],i))
################### Keep train till server end it ##############
while ml_contract.functions.loop().call()==True :
    while(ml_contract.functions.getIsStartTraining().call()==False):
        print("Wait for Server to Set isStartTraining to True")
        time.sleep(2)
    for i in range(len(users)):
        print(i)
        clients_obj[i].train_model()

    
##### Loop throw clients to init clients 


# user_id = '0'
# client = Client_model(data_path, user_id, contract_address)

0
Client 192.168.0.101 Training Started
12012
QmQLcdDbCqUzEJrxchJhW1qsyq9rkHmrPGf3H6mFqif1aj
Epoch 1/20


C:\Users\ahmed\anaconda3\envs\myenv\Lib\site-packages\keras\src\saving\saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


12012/12012 ━━━━━━━━━━━━━━━━━━━━ 97s 8ms/step - accuracy: 0.0017 - auc_3: 0.8704 - loss: 2.3205 - precision_3: 0.8850 - recall_3: 0.2215
Epoch 2/20
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 98s 8ms/step - accuracy: 1.8520e-04 - auc_3: 0.9182 - loss: 1.5036 - precision_3: 0.8098 - recall_3: 0.3159
Epoch 3/20
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 85s 7ms/step - accuracy: 7.4218e-04 - auc_3: 0.9250 - loss: 1.4400 - precision_3: 0.8033 - recall_3: 0.3498
Epoch 4/20
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 83s 7ms/step - accuracy: 0.0019 - auc_3: 0.9285 - loss: 1.4156 - precision_3: 0.8043 - recall_3: 0.3617
Epoch 5/20
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 83s 7ms/step - accuracy: 0.0016 - auc_3: 0.9305 - loss: 1.3955 - precision_3: 0.8104 - recall_3: 0.3691
Epoch 6/20
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 84s 7ms/step - accuracy: 0.0024 - auc_3: 0.9333 - loss: 1.3680 - precision_3: 0.8141 - recall_3: 0.3808
Epoch 7/20
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 86s 7ms/step - accuracy: 0.0042 - auc_3: 0.9340 - loss: 1.3578 - precis

C:\Users\ahmed\anaconda3\envs\myenv\Lib\site-packages\keras\src\saving\saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


12012/12012 ━━━━━━━━━━━━━━━━━━━━ 105s 9ms/step - accuracy: 0.0023 - auc_4: 0.8702 - loss: 2.3224 - precision_4: 0.8901 - recall_4: 0.2227
Epoch 2/20
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 103s 9ms/step - accuracy: 5.9833e-05 - auc_4: 0.9194 - loss: 1.4969 - precision_4: 0.8185 - recall_4: 0.3193
Epoch 3/20
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 103s 9ms/step - accuracy: 0.0016 - auc_4: 0.9267 - loss: 1.4350 - precision_4: 0.8102 - recall_4: 0.3504
Epoch 4/20
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 105s 9ms/step - accuracy: 0.0021 - auc_4: 0.9274 - loss: 1.4274 - precision_4: 0.8101 - recall_4: 0.3591
Epoch 5/20
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 107s 9ms/step - accuracy: 0.0030 - auc_4: 0.9288 - loss: 1.4118 - precision_4: 0.8116 - recall_4: 0.3645
Epoch 6/20
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 109s 9ms/step - accuracy: 0.0028 - auc_4: 0.9333 - loss: 1.3708 - precision_4: 0.8138 - recall_4: 0.3778
Epoch 7/20
12012/12012 ━━━━━━━━━━━━━━━━━━━━ 112s 9ms/step - accuracy: 0.0038 - auc_4: 0.9331 - loss: 1.3726 - pre

In [ ]:
ml_contract.functions.getTrainLoop().call()